# Project Development

Author: James Shaddix

## File Data File

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

class File_Data(object):
    """Stores all of the data associated with a single .csv or .xlsx file, in Dr. Ross's experiment"""
    def __init__(self, file_name, convert, signal_indices=[2,3], peak_indices=[0,1]):
        
        # Storing Parameters
        self.file_name      = file_name
        self.signal_indices = signal_indices
        self.peak_indices   = peak_indices
        self.convert        = convert
        
        # Synthesized Data (This Data Is Instantiated, when store_file_data is called)
        self.signal_headers, self.peak_headers = self._get_headers()
        self.x_signal = None
        self.y_signal = None
        self.x_peak = None
        self.y_peak = None
    
    def _get_headers(self):
        '''
        RETURNS: 2-[Lists] of 2-[Strings]
            * The file headers, corresponding to the indices that were passed in.
        '''
        df = pd.read_csv(self.file_name)
        headers = list(df)
        signal_headers = [headers[self.signal_indices[0]]] + \
                         [headers[self.signal_indices[1]]]
        peak_headers   = [headers[self.peak_indices[0]]]  + \
                         [headers[self.peak_indices[1]]]
               
        return signal_headers, peak_headers
    
    def store_file_data(self):
        '''
        DESCRIPTION:
            1. reads in all of the data, from all of the files
            2. splits the data into signal, and peak dataframes 
            3. parses all of the data
            4. stores headers, and all of the data associated with 
               a file in instance variables.
        RETURNS: [Dictionary]:
            - KEY:   [String] file_name
            - VALUE: [Tuple] of [dataframes] -> (signal-DataFrame, point-DataFrame)
        '''
        
        # Read Data
        df = pd.read_csv(read_file)
        
        # Store Headers
        self.signal_headers, self.peak_headers = self._get_headers()

        # Sepperate DataFrames
        df_signal  = df[self.signal_headers]
        df_peak    = df[self.peak_headers]

        # Drop Empty Rows
        df_signal = df_signal.dropna(axis=0) 
        df_peak   = df_peak.dropna(axis=0) 

        # Convert Data to Floats
        if convert is not None:
            df_signal = df_signal.applymap(convert)
            df_peak   = df_peak.applymap(convert)

        # Signal/Peaks to Plot
        self.x_signal = df_signal[self.signal_headers[0]]
        self.y_signal = df_signal[self.signal_headers[1]]
        self.x_peak  = df_peak[self.peak_headers[0]]
        self.y_peak  = df_peak[self.peak_headers[1]] 
        
        return 

In [ ]:
# test

def convert(val):
    '''
    DESCRIPTION:
        - converts a string into a float.
        - this function is passed into the Point_Click_Graph constructor
        - this function will be called on every data point in the data
          that is being analyzed by Point_Click_Graph
    '''
    
    # conversion dictionary
    conversions = {
                    'n':10**-9,
                    'u':10**-6,
                    'm':10**-3,
                    'k':10**3,  
                    'M':10**6
                  }
    
    # get the last character in a value
    str_val   = str(val)
    last_char = str_val[-1]
    
    # convert last char it is in dictionary if it is in conversion list
    if last_char in conversions:
        val = float(str_val[:-1])*conversions[last_char]
    
    # implictely convert to float
    else:     
        try:
            val = float(val)
        except:
            print("Failed to Implicitely Convert to Float: ",val)
            return None
    
    return val

read_file = "sp001_051518_BaIrO3_916_295K_0.1atm_100-1200kHz.csv"
fd = File_Data(read_file,convert)
fd.store_file_data()


plt.plot(fd.x_signal,fd.y_signal)
plt.plot(fd.x_peak,fd.y_peak,"ro")
plt.show()

## Point Click File

In [17]:
import matplotlib.pyplot as plt
import numpy as np

def click_plot(fig, ax, x_signal, y_signal, x_points, y_points, signal_kwargs={}):
    '''
    DESCRIPTION:
        - These plots have clickable points (The "point" data is clickable).
          Everytime a click is made on a point, the position of the point 
          is recorded. If the point is clicked on again, it will remove 
          the record of that point.
          
    RETURN: [List] of [List] containing 2_elements: (x-coordinate, y-coordinate)
            - The coordinates describe the locations that were recorded
    '''
    
    # Points to be Recorded
    important_points = []
    
    #      --- Plot The Data --- 
    
    # Signal Plot
    ax.plot(x_signal, y_signal, **signal_kwargs)

    # Point Plot
    coll = ax.scatter(x_points, y_points, color=["blue"]*len(x_points),
                      picker=5,label="Point Data")

    ax.legend(loc="best")
    
    #      --- Clickable Event ---
    
    def on_pick(event):

        # Index to Where The Event Occurred
        ind = event.ind[0]

        # Where The Event Occurred
        x_val, y_val = x_points[ind], y_points[ind]
       
        if [x_val,y_val] not in important_points:   
            
            # Record Point
            # color is speciified by: RGBA tuple
            # https://www.cgl.ucsf.edu/chimera/docs/ProgrammersGuide/Examples/footnotes/rgba.html
            important_points.append([x_val,y_val])
            coll._facecolors[ind,:] = (1, 0, 0, 1)
            coll._edgecolors[ind,:] = (1, 0, 0, 1)
            print("\t  Picked  Point [ {:>3} ] at: [ {:06.2f}, {:06.5f} ]".format(ind,x_val,y_val))
  
        else: 
        
            # Stop Recording Point
            important_points.remove([x_val,y_val])
            coll._facecolors[ind,:] = (0, 0, 1, 1)
            coll._edgecolors[ind,:] = (0, 0, 1, 1)
            print("\t  Removed Point [ {:>3} ] at: [ {:06.2f}, {:06.5f} ]".format(ind,x_val,y_val))

        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
    
    # Add Blocking to Stop The Program
    plt.show(block=True)

    return important_points

In [18]:
%matplotlib qt

def main():
    
    # Sample Data
    y = [0,0,0,2,0,0,0,-2,0,0,0,2,0,0,0,-2,0]
    x = range(len(y))
    
    y_peak = [2,-2,2,-2]
    x_peak = [3,7,11,15]

    # Set Plot Info
    fig, ax = plt.subplots()                
    ax.set_xlabel("x-axis")
    ax.set_ylabel("y-axis")
    ax.set_title("Click On The Peaks!")
    
    signal_kwargs = { 
                    "linewidth":1, 
                    "linestyle":"-",
                    "color":"green",
                    "label":"Magnitude vs. Frequency"
                    }

    # Make Clickable Plot
    imp = click_plot(fig, ax, x, y, x_peak, y_peak, signal_kwargs)
    
    print("\nPoints Picked:\n",imp)
    
if __name__ == "__main__":
    main()

	  Picked  Point [   0 ] at: [ 003.00, 2.00000 ]
	  Removed Point [   0 ] at: [ 003.00, 2.00000 ]
	  Picked  Point [   1 ] at: [ 007.00, -2.00000 ]
	  Removed Point [   1 ] at: [ 007.00, -2.00000 ]

Points Picked:
 []


## Controller File

In [ ]:
import subprocess
def get_files(directory='.'):
    '''
    Returns: 
        - list of csv files from a given directory
    Parameters:
        - Directory: specifies the directory with the files you would like to analyze.
            - By Defualt: this method will use the current directory.
    '''
    
    # use shell command to get file list
    str_files  = subprocess.check_output(["ls "+directory+" | grep '\.csv$'"],shell=True).decode("utf-8")
    list_files = str_files.split("\n")
    
    # remove empty list elements
    while '' in list_files: list_files.remove('')  
    
    # prepend the directory to each file_name
    list_files = [directory+'/'+file_name for file_name in list_files]
        
    return list_files

def convert(val):
    '''
    DESCRIPTION:
        - converts a string into a float.
        - this function is passed into the File_Data constructor
        - this function will be called on every data point in the data
          that is being analyzed by File_Data
    '''
    
    # conversion dictionary
    conversions = {
                    'n':10**-9,
                    'u':10**-6,
                    'm':10**-3,
                    'k':10**3,  
                    'M':10**6
                  }
    
    # get the last character in a value
    str_val   = str(val)
    last_char = str_val[-1]
    
    # convert last char it is in dictionary if it is in conversion list
    if last_char in conversions:
        val = float(str_val[:-1])*conversions[last_char]
    
    # implictely convert to float
    else:     
        try:
            val = float(val)
        except:
            print("Failed to Implicitely Convert to Float: ",val)
            return None
    
    return val

def get_temps(dict_imp_peaks):
    '''
    DESCRIPTION:
        * This method returns temperature data by parsing it from the files
        that are used.
        * This method is subject to change, if the file names are changed in the future.
    '''
    
    Temp = []
    for file_name in dict_imp_peaks:
        for peak in dict_imp_peaks[file_name]:
            Temp.append(float(file_name.split('_')[4][:-1]))
            
    return Temp

def important_peak_plot(dict_imp_peaks):
    '''
    DESCRIPTION:
        * Generates a composite plot, from all of the peaks that were clicked
        * Frequency vs. Temperature
    '''
    # get temperatures and frequencies to plot
    Temp  = get_temps(dict_imp_peaks)
    Freq  = []
    for file_name in dict_imp_peaks:
        for peak in dict_imp_peaks[file_name]:
            Freq.append(peak[0])

    # create plot
    fig = plt.figure()

    plt.plot(Temp,Freq,"o", 
             markeredgewidth=2,markeredgecolor='b',
             markerfacecolor='None',
             label="Frequency vs Temp"
            )

    plt.xlabel("Temperature [K]",fontsize=15)
    plt.ylabel("Freq [kHz]",fontsize=15)
    plt.title("$BalrO_{3}$ ba916 5/15/18",fontsize=20)
    plt.grid(True)

    plt.show()

def main():
    
    #      --- Plot Info ---

    # Set Default Plot Size
    mpl.rcParams['figure.figsize'] = (11,7) 


    #      --- Store The File Data and Plot ---

    # signal key-word-arguments for the plot
    signal_kwargs = { 
                    "linewidth":1, 
                    "linestyle":"-",
                    "color":"green",
                    "label":"Magnitude vs. Frequency"
                    }
    
    # get files
    data_files = get_files()

    for file in data_files:

        # store file data
        fd = File_Data(file,convert)
        fd.store_file_data()

        # set plot parameters
        fig, ax = plt.subplots()                
        ax.set_xlabel("Frequency")
        ax.set_ylabel("Voltage Magnitude")
        ax.set_title("Voltage vs. Frequency")
        ax.set_xlim(0,np.max(x_signal)+0.2*np.max(x_signal))
        ax.set_ylim(0,np.max(y_signal)+0.2*np.max(y_signal))
        
        # make interactive plot
        click_plot(ax, fd.x_signal, fd.y_signal, 
                       fd.x_peak,   fd.y_peak,
                   signal_kwargs)

if __name__ == "__main__":
    main()
